### collaborative_filtering.pyをインポートしてNMF後の行列から提出形式のファイルを出力するサンプル(カテゴリAの例)

- ユーザー×商品のnumpy配列はデータサイズが大きいので、dtypeをfloat16にする、使い終わったらdelするなどすると良いと思います。Aの場合はfloat64だと8byte×58658×13866で約6GB、float16だと約1.5GBになります

In [1]:
import numpy as np
import pandas as pd
import collaborative_filtering as cf

In [2]:
train = pd.read_table('../data/train/train_A.tsv')

In [3]:
#スコア行列の行に対応するユーザーIDのリスト
user_ids = train['user_id'].unique()
user_ids.sort()
#スコア行列の列に対応する商品IDのリスト
product_ids = train['product_id'].unique()
product_ids.sort()
#NMF後のスコア行列(ここでは例として乱数行列を用いてます)
mat = np.random.rand(len(user_ids), len(product_ids)).astype('float16')

print('user_ids')
print(user_ids)
print('product_ids')
print(product_ids)
print('mat')
print(mat)

user_ids
['0000000_A' '0000001_A' '0000002_A' ..., '0058655_A' '0058656_A'
 '0058657_A']
product_ids
['00000000_a' '00000001_a' '00000002_a' ..., '00014120_a' '00014121_a'
 '00014122_a']
mat
[[ 0.62695312  0.40820312  0.35913086 ...,  0.16674805  0.29785156
   0.30151367]
 [ 0.77099609  0.24951172  0.52734375 ...,  0.38867188  0.61376953
   0.86767578]
 [ 0.30249023  0.5234375   0.62841797 ...,  0.5859375   0.22753906
   0.52636719]
 ..., 
 [ 0.4128418   0.75        0.87646484 ...,  0.25805664  0.75927734
   0.00705338]
 [ 0.58789062  0.08526611  0.0914917  ...,  0.35302734  0.42456055
   0.66796875]
 [ 0.17871094  0.83349609  0.75097656 ...,  0.34838867  0.81054688
   0.66650391]]


In [4]:
#スコアを調整する行列
#(例えば、一度購入した商品はスコア行列を計算する際には高スコアでもレコメンドはしたくない、という際に
#一度購入した商品のスコアは-10点などとすればレコメンドはされなくなります。)
#ここでは一度購入した商品は-10点、他は0点(そのまま)としています
mat_add = ((cf.make_eventcountmat(data=train, event_type=3, users=user_ids, products=product_ids) != 0) * (-10)).astype('float16')

In [5]:
test = pd.read_table('../data/test.tsv')

In [8]:
#テストデータからユーザーidのカテゴリがAのものを取得
test_A = test[test['user_id'].map(lambda x: x[-1]) == 'A']
print('test_A')
test_A.head()

test_A


,user_id
0,0000008_A
1,0000719_A
2,0000720_A
3,0001355_A
4,0002685_A


In [9]:
#テストデータ、スコア行列、ユーザーIDリスト、商品IDリストからレコメンドを求める
submit_A = cf.make_recommend(test_A, mat + mat_add, user_ids, product_ids)
print('submit_A')
submit_A.head()

submit_A


,0,1,2
0,0000008_A,00012521_a,0
1,0000008_A,00005377_a,1
2,0000008_A,00002600_a,2
3,0000008_A,00008547_a,3
4,0000008_A,00008984_a,4


In [10]:
#提出形式で保存
submit_A.to_csv('submit_A.tsv', sep='\t', header=False, index=False, encoding='utf-8', line_terminator='\r\n')

In [11]:
#4カテゴリを合体するコード(もしくはメモ帳などでコピペ)
submit_A = pd.read_table('submit_A.tsv', header=None)
submit_B = pd.read_table('submit_B.tsv', header=None)
submit_C = pd.read_table('submit_C.tsv', header=None)
submit_D = pd.read_table('submit_D.tsv', header=None)
submit = pd.concat([submit_A, submit_B, submit_C, submit_D], axis=0)
submit.to_csv('submit.tsv', sep='\t', header=False, index=False, encoding='utf-8', line_terminator='\r\n')

### (NMFについて)

欠損値対応のNMFのやり方。

- scikit-learnのバージョンを最新(0.19.0)にする
- pythonのパッケージフォルダのsklearn内のdecomposition/nmf.py、decomposition/tests/test_nmf.py、utils/validation.pyをyasutake_everyoneフォルダ内のもので入れ替える

もしくは、上手くいかない、sklearnをいじりたくない場合は、交換済みのものをsklearn_newという名前で置いたのでフォルダごと作業ディレクトリに置いてインポートのsklearnをsklearn_newとすれば動くと思います。

In [13]:
from sklearn.decomposition import NMF

from numpy import nan as NA
#穴あき評価値行列
scoremat = np.array([
    [1, 2, 3, NA, NA],
    [4, 5, NA, 6, NA],
    [NA, 7, 8, 9, 1],
    [2, NA, 3, 4, 5]
])

#n_componentsはユーザーの特徴ベクトルの数なので、本番では100~1000くらいでしょうか?
model = NMF(n_components=2, init='random', random_state=1234, solver='mu', max_iter=200)

P = model.fit_transform(scoremat)
Q = model.components_
mat = np.dot(P, Q)

print('scoremat')
print(scoremat)
print('mat')
print(mat)

scoremat
[[  1.   2.   3.  nan  nan]
 [  4.   5.  nan   6.  nan]
 [ nan   7.   8.   9.   1.]
 [  2.  nan   3.   4.   5.]]
mat
[[  0.97508272   2.32630516   2.73074583   3.04327173   0.31441997]
 [  3.98845144   4.99852927   3.99916764   6.01006864  12.97764554]
 [  2.89913262   6.89090902   8.07829953   9.01167511   1.00136369]
 [  2.03457956   3.17586486   3.03386093   3.95812822   4.99972671]]
